The JokeGeneration workFlow in the 6.Persistance/1. shortTermMemory.ipynb.

So, we use the same code for the concept of TIME-TRAVEL : In which we go to any perivous checkpoint and then from that checkpoint GENERATE the flow..!!

In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

load_dotenv()

jokeLLM = ChatGoogleGenerativeAI(
  model='gemini-2.0-flash',
  temperature=0.7
)

class jokeState(TypedDict):
  topic:str
  joke:str
  explaination:str


def jokeGen(state:jokeState)->jokeState:
  prompt = f"""Write a funny, creative, and clean joke based on the topic: {state['topic']}. 
Make sure the joke is easy to understand, engaging, and suitable for a general audience."""

  response = jokeLLM.invoke(prompt).content
  return {'joke':response}


def jokeEx(state:jokeState)->jokeState:
  prompt = prompt = f"""
I have written a joke on the topic: "{state['topic']}".
Here is the joke: "{state['joke']}".

Please explain the joke in simple and clear terms so that anyone,
even someone unfamiliar with the topic, can understand why it is funny. 
Also highlight the wordplay, humor style (pun, irony, sarcasm, etc.), 
and the cultural or contextual background if relevant.
"""

  response = jokeLLM.invoke(prompt).content
  return {'explaination':response}


graph = StateGraph(jokeState)

graph.add_node('jokeGenerator', jokeGen)
graph.add_node('jokeExplaination', jokeEx)

graph.add_edge(START, 'jokeGenerator')
graph.add_edge('jokeGenerator', 'jokeExplaination')
graph.add_edge('jokeExplaination', END)

checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

config1 = {
  'configurable':{'thread_id':'1'}
}

config12 = {
  'configurable':{'thread_id':'12'}
}



c:\Users\Lenovo\Desktop\text\LangGraph\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### WorkFlow Execution

In [8]:
initalState = {
  'topic':'Mango'
}
workflow.invoke(initalState, config=config1)

{'topic': 'Mango',
 'joke': "Why did the mango break up with the coconut?\n\nBecause the coconut was too dry and the mango felt like they just couldn't *mang-go* on like that!",
 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is a play on words about why a mango and a coconut ended a relationship. The reason given is that the coconut was "too dry" (meaning boring or lacking in personality) and the mango felt they couldn\'t "mang-go" on.\n\n**Why it\'s funny:**\n\n*   **Wordplay (Pun):** The humor comes from the pun "mang-go," which sounds like "mango" but is also used as a substitute for the phrase "move on" or "continue" in a relationship. This creates a funny connection between the fruit and the relationship problem.\n\n*   **Humor Style (Pun):** The joke heavily relies on a pun, making it a pun-based joke.\n\n*   **Cultural/Contextual Background:** There isn\'t a strong cultural or contextual background necessary to understand the joke.

In [9]:
initalState = {
  'topic':'Pineapple'
}
workflow.invoke(initalState, config=config12)

{'topic': 'Pineapple',
 'joke': 'Why did the pineapple break up with the coconut?\n\nBecause he said she was too clingy, and he needed some *space* between them... ya know, like the space between the spikes on his head! He said, "You\'re coconuts for me, but I need my room!"',
 'explaination': 'Okay, let\'s break down the pineapple joke:\n\n**Explanation:**\n\nThe joke is about a pineapple breaking up with a coconut. The humor comes from personifying these fruits and giving them human-like relationship problems. The punchline explains the reason for the breakup: the pineapple feels the coconut is too attached and needs more personal space. This is then linked back to the physical characteristics of the pineapple – its spiky exterior.\n\n**Why it\'s funny:**\n\n*   **Personification:** The joke\'s basic premise is funny because it gives fruits human characteristics and relationship issues. We don\'t normally think of pineapples and coconuts having romantic relationships.\n*   **Relatabi

## ChatHistroy

chatHistroy of config1

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Mango', 'joke': "Why did the mango break up with the coconut?\n\nBecause the coconut was too dry and the mango felt like they just couldn't *mang-go* on like that!", 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is a play on words about why a mango and a coconut ended a relationship. The reason given is that the coconut was "too dry" (meaning boring or lacking in personality) and the mango felt they couldn\'t "mang-go" on.\n\n**Why it\'s funny:**\n\n*   **Wordplay (Pun):** The humor comes from the pun "mang-go," which sounds like "mango" but is also used as a substitute for the phrase "move on" or "continue" in a relationship. This creates a funny connection between the fruit and the relationship problem.\n\n*   **Humor Style (Pun):** The joke heavily relies on a pun, making it a pun-based joke.\n\n*   **Cultural/Contextual Background:** There isn\'t a strong cultural or contextual background necessary to 

chatHistroy of config12

In [14]:
list(workflow.get_state_history(config12))

[StateSnapshot(values={'topic': 'Pineapple', 'joke': 'Why did the pineapple break up with the coconut?\n\nBecause he said she was too clingy, and he needed some *space* between them... ya know, like the space between the spikes on his head! He said, "You\'re coconuts for me, but I need my room!"', 'explaination': 'Okay, let\'s break down the pineapple joke:\n\n**Explanation:**\n\nThe joke is about a pineapple breaking up with a coconut. The humor comes from personifying these fruits and giving them human-like relationship problems. The punchline explains the reason for the breakup: the pineapple feels the coconut is too attached and needs more personal space. This is then linked back to the physical characteristics of the pineapple – its spiky exterior.\n\n**Why it\'s funny:**\n\n*   **Personification:** The joke\'s basic premise is funny because it gives fruits human characteristics and relationship issues. We don\'t normally think of pineapples and coconuts having romantic relationsh

## Time Travel

Get the checkpoint_id of the thread_id = '1' & pass it in to the thread-id dict

In [15]:
workflow.get_state({'configurable':{'thread_id':'1', 'checkpoint_id':'1f08a1b6-c851-6a62-8000-f63d5ee0383f'}})

StateSnapshot(values={'topic': 'Mango'}, next=('jokeGenerator',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f08a1b6-c851-6a62-8000-f63d5ee0383f'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-05T05:44:43.789745+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a1b6-c84a-6a41-bfff-8c9f73ff09bf'}}, tasks=(PregelTask(id='e4d2715c-9b52-ff88-c009-c4762609f26e', name='jokeGenerator', path=('__pregel_pull', 'jokeGenerator'), error=None, interrupts=(), state=None, result={'joke': "Why did the mango break up with the coconut?\n\nBecause the coconut was too dry and the mango felt like they just couldn't *mang-go* on like that!"}),), interrupts=())

In [16]:
workflow.invoke(None, {'configurable':{'thread_id':'1', 'checkpoint_id':'1f08a1b6-c851-6a62-8000-f63d5ee0383f'}})

{'topic': 'Mango',
 'joke': 'Why did the mango break up with the pineapple?\n\nBecause it said, "I can\'t stand your prickly personality anymore! I need someone a little less... **mango-nificent**!"',
 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is a play on words related to a romantic breakup. The humor comes from replacing a more common adjective with a word that sounds similar but is related to the topic (mango). The mango is breaking up with the pineapple because it finds the pineapple\'s personality unpleasant ("prickly"). Instead of saying it needs someone "more magnificent," it says "more mango-nificent," which is a silly and unexpected thing to say.\n\n**Humor Analysis:**\n\n*   **Wordplay (Pun):** The core of the joke is the pun on "magnificent" and "mango-nificent." It uses the word "mango" in a context where it doesn\'t logically belong, creating a humorous effect.\n*   **Character Stereotype:** The joke relies on a slight st

In [18]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Mango', 'joke': 'Why did the mango break up with the pineapple?\n\nBecause it said, "I can\'t stand your prickly personality anymore! I need someone a little less... **mango-nificent**!"', 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is a play on words related to a romantic breakup. The humor comes from replacing a more common adjective with a word that sounds similar but is related to the topic (mango). The mango is breaking up with the pineapple because it finds the pineapple\'s personality unpleasant ("prickly"). Instead of saying it needs someone "more magnificent," it says "more mango-nificent," which is a silly and unexpected thing to say.\n\n**Humor Analysis:**\n\n*   **Wordplay (Pun):** The core of the joke is the pun on "magnificent" and "mango-nificent." It uses the word "mango" in a context where it doesn\'t logically belong, creating a humorous effect.\n*   **Character Stereotype:** The joke r

## Update State



In [20]:
workflow.update_state({'configurable':{'thread_id':'1', 'checkpoint_id':'1f08a1b6-c84a-6a41-bfff-8c9f73ff09b'}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f08a1e1-3229-6557-8000-0572c5f8de2a'}}

In [21]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('jokeGenerator',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a1e1-3229-6557-8000-0572c5f8de2a'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-09-05T06:03:42.317052+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08a1b6-c84a-6a41-bfff-8c9f73ff09b'}}, tasks=(PregelTask(id='1c5a5bc9-2505-160f-f960-7fdf42c16e8b', name='jokeGenerator', path=('__pregel_pull', 'jokeGenerator'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'Mango', 'joke': 'Why did the mango break up with the pineapple?\n\nBecause it said, "I can\'t stand your prickly personality anymore! I need someone a little less... **mango-nificent**!"', 'explaination': 'Okay, here\'s a breakdown of the mango joke:\n\n**Explanation:**\n\nThe joke is a play on words related to a romantic breakup. The humor comes

### 1. Update state invoke

##### 1.1 generate the new joke from the updated topic , need new topic checkpoint_id

In [23]:
workflow.invoke(None, {'configurable':{'thread_id':'1', 'checkpoint_id':'1f08a1e1-3229-6557-8000-0572c5f8de2a'}})

{'topic': 'samosa',
 'joke': 'Why did the samosa break up with the chutney?\n\nBecause it said, "I need some space! You\'re always all over me, and frankly, I\'m starting to feel a little… filled!"',
 'explaination': 'Here\'s a breakdown of the samosa joke:\n\n**Explanation:**\n\nThe joke is funny because it personifies a samosa and chutney (a common dipping sauce for samosas) as being in a relationship. The humor comes from applying relationship problems to food. The samosa wants to break up because the chutney is always "all over" it, meaning it\'s constantly being dipped into the chutney. The samosa then says it\'s feeling "filled," which is a double meaning:\n\n*   **Literal meaning:** Samosas are filled with potatoes, peas, and spices. The samosa is already full of these ingredients.\n*   **Figurative meaning:** The samosa is "filled" with the chutney, meaning it\'s overwhelmed and doesn\'t want any more.\n\n**Wordplay:**\n\n*   **"Filled":** This is the key pun in the joke. It us

we have total = 9 Histroy

1. last 4 invoke is inital invoke
2. next 2 is time travel for excuting with the checkpoint_id
3. next 1 is update the state
4. nd the first 2 which is the invoke with the update state

In [25]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa break up with the chutney?\n\nBecause it said, "I need some space! You\'re always all over me, and frankly, I\'m starting to feel a little… filled!"', 'explaination': 'Here\'s a breakdown of the samosa joke:\n\n**Explanation:**\n\nThe joke is funny because it personifies a samosa and chutney (a common dipping sauce for samosas) as being in a relationship. The humor comes from applying relationship problems to food. The samosa wants to break up because the chutney is always "all over" it, meaning it\'s constantly being dipped into the chutney. The samosa then says it\'s feeling "filled," which is a double meaning:\n\n*   **Literal meaning:** Samosas are filled with potatoes, peas, and spices. The samosa is already full of these ingredients.\n*   **Figurative meaning:** The samosa is "filled" with the chutney, meaning it\'s overwhelmed and doesn\'t want any more.\n\n**Wordplay:**\n\n*   **"Filled":** This is the key pu